# Light GBM with cross validation

In [4]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

In [1]:
def lgb_evaluate(num_boost_round, learning_rate,max_depth,reg_alpha,reg_lambda,subsample,colsample_bytree,
                min_child_weight,num_leaves):
        
    clf = lgb.LGBMClassifier(
        num_boost_round= int(num_boost_round),
        learning_rate= learning_rate,
        max_depth= int(max_depth),
        reg_alpha= reg_alpha,
        reg_lambda=reg_lambda,
        subsample= subsample,
        colsample_bytree=colsample_bytree,
        min_child_weight=min_child_weight,
        n_jobs=-1,
        random_seed=2021,
        objective = 'binary', 
        num_leaves= int(num_leaves),metric= 'auc',
        verbose =-1
    )
    
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1')
    print(np.mean(scores))

    return np.mean(scores)

In [2]:
def bayesOpt(X_train, y_train):
    lgbBO = BayesianOptimization(lgb_evaluate, {'num_leaves': (10,50),
                                                'max_depth': (3,5),
                                                'learning_rate': (0.1, 0.3),
                                                'num_boost_round': (25, 200),
                                                'reg_alpha' : (0.1, 1),
                                                'reg_lambda': (0.1, 1),
                                                'subsample': (0.5,  0.9),
                                                'colsample_bytree': (0.5,1),
                                                'min_child_weight': (0,10),
                                               })


    lgbBO.maximize(init_points=5, n_iter=5)

    print(lgbBO.max)


Call the function "bayesOpt(X_train, y_train)" and the best parameters will be printed. Pass those parameters in LGBMCLassifier in the next cell.

In [3]:
lgbm_clf_bayesian_cross = LGBMClassifier(colsample_bytree=0.6296708043894408, learning_rate= 0.13134455240151333,
                                   max_depth=3, min_child_weight=6.365296419412429, n_estimators=182,
                                   subsample=0.899447424209072, objective='binary', random_state= 2021,
                          boosting_type ='gbdt', reg_lambda= 0.35105827634182885, reg_alpha= 0.15100610009471693,
                                  num_leaves=45)

lgbm_clf_bayesian_cross.fit(X_train, y_train)

predictions_train = lgbm_clf_bayesian_cross.predict(X_train)
predictions_test = lgbm_clf_bayesian_cross.predict(X_test)

###Train data 
print("Confusion Matrix for train data:")
print(confusion_matrix(y_train, predictions_train))
print('\n')
print("Classification Report for train data")
print(classification_report(y_train, predictions_train))
print('\n')


###Test data
print("Confusion Matrix for test data:")
print(confusion_matrix(y_test, predictions_test))
print('\n')
print("Classification Report for test data")
print(classification_report(y_test, predictions_test))

NameError: name 'LGBMClassifier' is not defined

# XGBoost with Cross Validation

In [5]:
def xgb_evaluate(n_estimators, learning_rate,max_depth,reg_alpha,reg_lambda,subsample,colsample_bytree,
                min_child_weight,gamma):
        
    xgb_1 = xgb.XGBClassifier(
        n_estimators= int(n_estimators),
        learning_rate= learning_rate,
        max_depth= int(max_depth),
        reg_alpha= reg_alpha,
        reg_lambda=reg_lambda,
        subsample= subsample,
        colsample_bytree=colsample_bytree,
        min_child_weight=min_child_weight,
        n_jobs=-1,
        random_seed=2021,
        objective = 'binary:logistic', 
        gamma= gamma,metric= 'auc',
        verbose =-1
    )
    
    scores = cross_val_score(xgb_1, X_train, y_train, cv=5, scoring='f1')
    print(np.mean(scores))

    return np.mean(scores)

In [6]:
def bayesOpt(X_train, y_train):
    lgbBO = BayesianOptimization(xgb_evaluate, {'max_depth': (3,5),
                                                'learning_rate': (0.1, 0.3),
                                                'n_estimators': (25, 200),
                                                'reg_alpha' : (0.1, 1),
                                                'reg_lambda': (0.1, 1),
                                                'subsample': (0.5,  0.9),
                                                'colsample_bytree': (0.5,1),
                                                'min_child_weight': (0,10),
                                                'gamma': (0,1)
                                               })


    lgbBO.maximize(init_points=5, n_iter=5)

    print(lgbBO.max)

In [7]:
from xgboost import XGBClassifier
model_xgb= XGBClassifier()

Call the bayesOpt(X_train, y_train) and Pass the best paramaters into the XGBClassifier() as shown below

In [ ]:
xgb_clf_bayesian_cross = XGBClassifier(colsample_bytree=0.7175669183131084, learning_rate= 0.16996462009544472,
                                   max_depth=4, min_child_weight=0.15148888091017665, n_estimators=153,
                                   subsample=0.8559299578122153, random_state= 2021,
                          boosting_type ='gbdt', reg_lambda= 0.6149041547479859, reg_alpha= 0.813321397675256,
                                  gamma=0.7884315900628941)

xgb_clf_bayesian_cross.fit(X_train, y_train)

predictions_train = xgb_clf_bayesian_cross.predict(X_train)
predictions_test = xgb_clf_bayesian_cross.predict(X_test)

###Train data 
print("Confusion Matrix for train data:")
print(confusion_matrix(y_train, predictions_train))
print('\n')
print("Classification Report for train data")
print(classification_report(y_train, predictions_train))
print('\n')

###Validation data
# print("Confusion Matrix for validation data:")
# print(confusion_matrix(y_val, predictions_val))
# print('\n')
# print("Classification Report for validation data")
# print(classification_report(y_val, predictions_val))
# print('\n')

###Test data
print("Confusion Matrix for test data:")
print(confusion_matrix(y_test, predictions_test))
print('\n')
print("Classification Report for test data")
print(classification_report(y_test, predictions_test))

In [8]:
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, roc_auc_score
def auprc(y,preds):
    precision, recall, thresholds = precision_recall_curve(y,preds)
    avg_prec = average_precision_score(y,preds)
    plt.plot(recall,precision, label='avg precision='+str(avg_prec))
    plt.legend(loc=4)
    
    fpr, tpr, _ = roc_curve(y, preds)
    auc = roc_auc_score(y, preds)
    plt.plot(fpr, tpr, label="auc"+str(auc))
    plt.legend(loc=4)
    
    plt.show()

In [ ]:
from pprint import pprint
train_pred_confusion = [1 if w>=0.4 else 0 for w in list(train_preds)]
print("Confusion Matrix: ")
print(confusion_matrix(y_train, train_pred_confusion),'\n')
pprint(classification_report(y_train, train_pred_confusion))

test_pred_confusion = [1 if w>=0.4 else 0 for w in list(test_preds)]
print("Confusion Matrix: ")
print(confusion_matrix(y_test, test_pred_confusion),'\n')
pprint(classification_report(y_test, test_pred_confusion))